# Load Reddit Data

In [1]:
import pickle
import pandas as pd

with open('../data/allSubredditsEnriched.pickle','rb') as read_file:
    all_subreddits = pickle.load(read_file)

# Define "Core" Network

In [2]:
network_seed_list = [
'r/latinoamerica',
'r/argentina',
'r/bolivia',
'r/brasil',
'r/chile',
'r/colombia',
'r/costa_rica',
'r/cuba',
'r/dominican',
'r/ecuador',
'r/estadosunidos',
'r/elsalvador',
'r/es',
'r/guatemala',
'r/honduras',
'r/mexico',
'r/nicaragua',
'r/panama',
'r/paraguay',
'r/peru',
'r/puertorico',
'r/spain',
'r/uruguay',
'r/venezuela',
'r/LatinAmerica',
'r/redditores',
'r/espanol',
'r/catolicismo',
'r/futbol',
'r/videojuego',
'r/musicaenespanol',
]

# Find Out Degree Nodes

In [ ]:
import re

with_word_breaks = list(map(lambda w: r'\b' + w + r'\b', network_seed_list))
regex = re.compile('|'.join(with_word_breaks))

def filter_network_reddits(dataframe):
    return dataframe[dataframe['description_html'].str.contains(regex, na=False) | dataframe['wiki_text'].str.contains(regex, na=False)]

In [4]:
out_degree = filter_network_reddits(all_subreddits)

# Create Edges and Attribute List

In [12]:
formatted = out_degree.reset_index().rename(columns={ 'real_name': 'reddit_name' })

In [13]:
import sys
import importlib
sys.path.append('../src/')

import reddit
reddit = importlib.reload(reddit)

import network
network = importlib.reload(network)

reddit_list = list(formatted['reddit_name'])
edges = list(map(lambda name: network.find_edges(reddit_name=name, reddit_type='r', dataframe=all_subreddits, depth='out_degree'), reddit_list))

In [23]:
import pandas as pd

flattened_edges = [y for x in edges for y in x]
df = pd.DataFrame(flattened_edges)

In [28]:
edges_to_core_network = df[df['reddit'].isin(list(map(lambda s: s[2:], network_seed_list)))]

In [38]:
renamed = edges_to_core_network.rename(columns={ 'parent': 'source', 'type_of_parent': 'source_type', 'reddit': 'target', 'type_of_reddit': 'target_type', 'type_of_edge': 'edge_type' })
deduped = renamed.drop_duplicates(subset=['source', 'target', 'edge_type'])
deduped[['source', 'source_type', 'target', 'target_type', 'edge_type', 'context']].to_csv('../data/outDegreeEdgeList.csv')

In [41]:
reddits_in_edge_list = formatted[formatted['reddit_name'].isin(edges_to_core_network.source.unique())]
reddits_in_edge_list[['reddit_name', 'desc', 'created_date', 'subs']].to_csv('../data/outDegreeAttributeList.csv')